<a href="https://colab.research.google.com/github/ariflaksito/recsys/blob/master/01_cb_tmdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content-based Filtering
Contoh kode pada teknik content-based filtering pada sistem rekomendasi



#Import library

In [0]:
import pandas as pd
import numpy as np

#Import Data
Data movie diambil dari:<br> https://www.kaggle.com/tmdb/tmdb-movie-metadata

In [0]:
credits = pd.read_csv('/content/drive/My Drive/Colab Data/tmdb_5000_credits.csv')
credits.head()

In [0]:
movies = pd.read_csv('/content/drive/My Drive/Colab Data/tmdb_5000_movies.csv')
movies.head()

In [0]:
print("Credits", credits.shape)
print("Movies", movies.shape)

#Gabung Data
Merge/gabung data credits & movies <br>
Perlu disesuaikan id pada data credits & movies

In [0]:
credits_col_renamed = credits.rename(index=str, columns={"movie_id":"id"})
movies_df_merge = movies.merge(credits_col_renamed, on="id")
movies_df_merge.head()

#Clean Data

In [0]:
movies_df_cleaned = movies_df_merge.drop(columns=["homepage","title_x","title_y","status","production_countries"])
movies_df_cleaned.head()

In [0]:
movies_df_cleaned.info()

# Mengukur nilai rata-rata untuk peringkat movies
Membuat rekomendasi berdasarkan deskripsi/ringkasan movie yang tersedia. Tujuan rekomendasi ini adalah memberikan list movie yang memiliki kesamaan ringkasan

In [0]:
movies_df_cleaned.head(1)['overview']

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3, max_features=None,
                      strip_accents='unicode', analyzer='word',
                      token_pattern=r'\w{1,}',ngram_range=(1,3),
                      stop_words= 'english')

# Filling NaNs with empty string
movies_df_cleaned['overview'] = movies_df_cleaned['overview'].fillna('')

# Fitting the TF-IDF on 'overview' text
tfv_matrix = tfv.fit_transform(movies_df_cleaned['overview'])

In [0]:
tfv_matrix

<4803x10417 sparse matrix of type '<class 'numpy.float64'>'
	with 127220 stored elements in Compressed Sparse Row format>

In [0]:
tfv_matrix.shape

In [0]:
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [0]:
sig[0]

In [0]:
# Reverse mapping of indices and movie title
indices = pd.Series(movies_df_cleaned.index, index=movies_df_cleaned['original_title']).drop_duplicates()

In [0]:
indices

In [0]:
sig[4799]

array([0.76159416, 0.76159416, 0.76159438, ..., 0.76159432, 0.76159416,
       0.76159478])